<a href="https://colab.research.google.com/github/itsanuragkumarjha/RAG_implemented_pdf_chatbot/blob/main/athina_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers faiss-cpu PyMuPDF streamlit rouge nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


In [2]:
import fitz  # PyMuPDF
from transformers import DPRQuestionEncoderTokenizer, DPRQuestionEncoder
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
import faiss
import numpy as np
import streamlit as st


In [4]:
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text += page.get_text("text")
    return text

# Load the PDF and extract text
pdf_path = "/content/policy-booklet-0923.pdf"
pdf_text = extract_text_from_pdf(pdf_path)
print("PDF text extracted.")


PDF text extracted.


In [5]:
def chunk_text(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

chunks = chunk_text(pdf_text)
print(f"Text chunked into {len(chunks)} chunks.")


Text chunked into 107 chunks.


In [6]:
retriever_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
retriever = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
generator_tokenizer = T5Tokenizer.from_pretrained('t5-base')
generator = T5ForConditionalGeneration.from_pretrained('t5-base')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
def index_chunks(chunks):
    index = faiss.IndexFlatL2(768)  # Assuming 768-dimensional embeddings
    chunk_embeddings = []
    for chunk in chunks:
        inputs = retriever_tokenizer(chunk, return_tensors='pt', padding=True, truncation=True)
        chunk_embedding = retriever(**inputs).pooler_output.detach().numpy()
        chunk_embeddings.append(chunk_embedding)
    chunk_embeddings = np.vstack(chunk_embeddings)
    index.add(chunk_embeddings)
    return index, chunk_embeddings

index, chunk_embeddings = index_chunks(chunks)
print("Chunks indexed with FAISS.")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Chunks indexed with FAISS.


In [8]:
def get_answer(query, chunks, index, chunk_embeddings):
    inputs = retriever_tokenizer(query, return_tensors='pt')
    question_embedding = retriever(**inputs).pooler_output.detach().numpy()
    distances, indices = index.search(question_embedding, 1)
    retrieved_chunk = chunks[indices[0][0]]
    input_ids = generator_tokenizer(retrieved_chunk, return_tensors='pt').input_ids
    output_ids = generator.generate(input_ids)
    answer = generator_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Test the get_answer function
query = "What is the contact number for Churchill customer service?"
print(f"Question: {query}")
print(f"Answer: {get_answer(query, chunks, index, chunk_embeddings)}")


Question: What is the contact number for Churchill customer service?


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Answer: a free car insurance quote. Page 43 If you’re in an accident Need to


In [9]:
def evaluate_model(dataset, chunks, index, chunk_embeddings):
    y_true = []
    y_pred = []

    for item in dataset:
        query = item['question']
        true_answer = item['answer']
        predicted_answer = get_answer(query, chunks, index, chunk_embeddings)

        y_true.append(true_answer)
        y_pred.append(predicted_answer)

    bleu_scores = [sentence_bleu([true], pred) for true, pred in zip(y_true, y_pred)]
    rouge = Rouge()
    rouge_scores = rouge.get_scores(y_pred, y_true, avg=True)

    return {
        "BLEU": sum(bleu_scores) / len(bleu_scores),
        "ROUGE": rouge_scores,
    }

# Load the QA dataset
with open("/content/qa_dataset.json", 'r') as f:
    dataset = json.load(f)

# Evaluate the model
evaluation_results = evaluate_model(dataset, chunks, index, chunk_embeddings)
print("Evaluation Results:", evaluation_results)


Evaluation Results: {'BLEU': 0.12065370912529809, 'ROUGE': {'rouge-1': {'r': 0.07631962551746377, 'p': 0.10917845770786946, 'f': 0.08612043342828961}, 'rouge-2': {'r': 0.009542340327944042, 'p': 0.015300059417706477, 'f': 0.011360846959803803}, 'rouge-l': {'r': 0.07143427524208791, 'p': 0.10021876271876269, 'f': 0.07994629710709447}}}


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [5]:
!wget -q -O - ipv4.icanhazip.com


34.75.56.164


In [15]:
# Streamlit front-end
%%writefile app.py
import streamlit as st
st.title("RAG-Powered PDF Chatbot")

user_query = st.text_input("Enter your question:")
if user_query:
    answer = get_answer(user_query, chunks, index, chunk_embeddings)
    st.write("Answer:", answer)


Overwriting app.py


In [17]:
%%writefile App.py
import fitz  # PyMuPDF
from transformers import DPRQuestionEncoderTokenizer, DPRQuestionEncoder
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
import faiss
import numpy as np
import streamlit as st

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text += page.get_text("text")
    return text

# Function to chunk text into smaller segments
def chunk_text(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Initialize models
retriever_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
retriever = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
generator_tokenizer = T5Tokenizer.from_pretrained('t5-base')
generator = T5ForConditionalGeneration.from_pretrained('t5-base')

# Index chunks using FAISS
def index_chunks(chunks):
    index = faiss.IndexFlatL2(768)  # Assuming 768-dimensional embeddings
    chunk_embeddings = []
    for chunk in chunks:
        inputs = retriever_tokenizer(chunk, return_tensors='pt', padding=True, truncation=True)
        chunk_embedding = retriever(**inputs).pooler_output.detach().numpy()
        chunk_embeddings.append(chunk_embedding)
    chunk_embeddings = np.vstack(chunk_embeddings)
    index.add(chunk_embeddings)
    return index, chunk_embeddings

# Function to get answer to a query
def get_answer(query, chunks, index, chunk_embeddings):
    # Encode query using retriever
    inputs = retriever_tokenizer(query, return_tensors='pt')
    question_embedding = retriever(**inputs).pooler_output.detach().numpy()

    # Search for the most relevant chunk
    distances, indices = index.search(question_embedding, 1)
    retrieved_chunk = chunks[indices[0][0]]

    # Generate answer using generator
    input_ids = generator_tokenizer(retrieved_chunk, return_tensors='pt').input_ids
    output_ids = generator.generate(input_ids)
    answer = generator_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return answer

# Evaluation function
def evaluate_model(dataset, chunks, index, chunk_embeddings):
    y_true = []
    y_pred = []

    for item in dataset:
        query = item['question']
        true_answer = item['answer']
        predicted_answer = get_answer(query, chunks, index, chunk_embeddings)

        y_true.append(true_answer)
        y_pred.append(predicted_answer)

    # Calculate BLEU and ROUGE scores
    bleu_scores = [sentence_bleu([true], pred) for true, pred in zip(y_true, y_pred)]
    rouge = Rouge()
    rouge_scores = rouge.get_scores(y_pred, y_true, avg=True)

    return {
        "BLEU": sum(bleu_scores) / len(bleu_scores),
        "ROUGE": rouge_scores,
    }

# Load dataset
with open("/content/qa_dataset.json", 'r') as f:
    dataset = json.load(f)

# Example usage (for testing purposes)
if __name__ == "__main__":
    pdf_text = extract_text_from_pdf("/content/policy-booklet-0923.pdf")
    chunks = chunk_text(pdf_text)
    index, chunk_embeddings = index_chunks(chunks)
    evaluation_results = evaluate_model(dataset, chunks, index, chunk_embeddings)
    print("Evaluation Results:", evaluation_results)

    # Streamlit front-end
    st.title("RAG-Powered PDF Chatbot")

    user_query = st.text_input("Enter your question:")
    if user_query:
        answer = get_answer(user_query, chunks, index, chunk_embeddings)
        st.write("Answer:", answer)


Writing App.py


In [18]:
!streamlit run App.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.185.189.227:8501

npx: installed 22 in 3.325s
your url is: https://cool-parks-smell.loca.lt
Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You are using the default legacy behavio

In [6]:
%%writefile app2.py
import fitz  # PyMuPDF
from transformers import DPRQuestionEncoderTokenizer, DPRQuestionEncoder
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
import faiss
import numpy as np
import streamlit as st

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text += page.get_text("text")
    return text

# Function to chunk text into smaller segments
def chunk_text(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Initialize models
retriever_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
retriever = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
generator_tokenizer = T5Tokenizer.from_pretrained('t5-base')
generator = T5ForConditionalGeneration.from_pretrained('t5-base')

# Index chunks using FAISS
def index_chunks(chunks):
    index = faiss.IndexFlatL2(768)  # Assuming 768-dimensional embeddings
    chunk_embeddings = []
    for chunk in chunks:
        inputs = retriever_tokenizer(chunk, return_tensors='pt', padding=True, truncation=True)
        chunk_embedding = retriever(**inputs).pooler_output.detach().numpy()
        chunk_embeddings.append(chunk_embedding)
    chunk_embeddings = np.vstack(chunk_embeddings)
    index.add(chunk_embeddings)
    return index, chunk_embeddings

# Function to get answer to a query
def get_answer(query, chunks, index, chunk_embeddings, max_length=50):
    # Encode query using retriever
    inputs = retriever_tokenizer(query, return_tensors='pt')
    question_embedding = retriever(**inputs).pooler_output.detach().numpy()

    # Search for the most relevant chunk
    distances, indices = index.search(question_embedding, 1)
    retrieved_chunk = chunks[indices[0][0]]

    # Generate answer using generator
    input_ids = generator_tokenizer(retrieved_chunk, return_tensors='pt').input_ids
    output_ids = generator.generate(input_ids, max_length=max_length)
    answer = generator_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return answer

# Load and process PDF
pdf_text = extract_text_from_pdf("/content/policy-booklet-0923.pdf")
chunks = chunk_text(pdf_text)
index, chunk_embeddings = index_chunks(chunks)

# Streamlit front-end
st.title("RAG-Powered PDF Chatbot")

user_query = st.text_input("Enter your question:")
if user_query:
    answer = get_answer(user_query, chunks, index, chunk_embeddings, max_length=100)
    st.write("Answer:", answer)


Overwriting app2.py


In [7]:
!streamlit run app2.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.75.56.164:8501

npx: installed 22 in 2.369s
your url is: https://tasty-cycles-jam.loca.lt
tokenizer_config.json: 100% 28.0/28.0 [00:00<00:00, 152kB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 3.74MB/s]
tokenizer.json: 100% 466k/466k [00:00<00:00, 9.38MB/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
config.json: 100% 493/493 [00:00<00:00, 2.81MB/s]
pytorch_model.bin: 100% 438M/438M [00:03<00:00, 134MB/s]
Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'questi